In [ ]:
import os
import time
import tensorflow as tf
import setproctitle as SP
import pandas as pd
import numpy as np
import datetime
import csv
from pandas import DataFrame
pd.options.display.max_rows = None
pd.options.display.max_columns = None
import matplotlib.pyplot as plt
import scipy.io
from sklearn.model_selection import train_test_split
from numpy import savetxt
%matplotlib notebook

import os
import time
import tensorflow as tf
import setproctitle as SP
import pandas as pd
import numpy as np
import datetime
import csv
from pandas import DataFrame
pd.options.display.max_rows = None
pd.options.display.max_columns = None
import matplotlib.pyplot as plt
import scipy.io
from sklearn.model_selection import train_test_split
from numpy import savetxt
    
import mpl_toolkits.mplot3d

# Functions from @Mateen Ulhaq and @karlo
def set_axes_equal(ax: plt.Axes):
    """Set 3D plot axes to equal scale.

    Make axes of 3D plot have equal scale so that spheres appear as
    spheres and cubes as cubes.  Required since `ax.axis('equal')`
    and `ax.set_aspect('equal')` don't work on 3D.
    """
    limits = np.array([
        ax.get_xlim3d(),
        ax.get_ylim3d(),
        ax.get_zlim3d(),
    ])
    origin = np.mean(limits, axis=1)
    radius = 0.5 * np.max(np.abs(limits[:, 1] - limits[:, 0]))
    _set_axes_radius(ax, origin, radius)

def _set_axes_radius(ax, origin, radius):
    x, y, z = origin
    ax.set_xlim3d([x - radius, x + radius])
    ax.set_ylim3d([y - radius, y + radius])
    ax.set_zlim3d([z - radius, z + radius])


In [ ]:
# tf.config.list_physical_devices()
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.1)

sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))


In [ ]:
# os.chdir("/tf/tf/NaturePaper")
os.chdir("/data/raw_data")


ScanTime='10-05-2022 15.56.10.800'
CoordinateRaw = pd.read_csv('Coordinate_matched_'+ScanTime+'.csv')
SpectraRaw = pd.read_csv('Spectrum_matched_'+ScanTime+'.csv')

# print(type(CoordinateRaw))
# print(CoordinateRaw.shape)
# print(CoordinateRaw.iloc[0,42])
# print(CoordinateRaw.iloc[0,:])

print(type(SpectraRaw))
print(SpectraRaw.shape)
# print(SpectraRaw.iloc[0,42])
# print(SpectraRaw.iloc[0,:])
print(SpectraRaw.iloc[0:10,0])




In [ ]:
print(type(CoordinateRaw))
print(CoordinateRaw.shape)
print(CoordinateRaw.iloc[0,42])
print(CoordinateRaw.iloc[0,:])

In [ ]:
Spectra=SpectraRaw
Coordinate=CoordinateRaw
Spectra.iloc[:,0] =  pd.to_datetime(SpectraRaw.iloc[:,0], format='%Y-%m-%d %H:%M:%S.%f')
Spectra=Spectra.set_index('timestamps')
Coordinate.iloc[:,0] =  pd.to_datetime(CoordinateRaw.iloc[:,0], format='%Y-%m-%d %H:%M:%S.%f')
Coordinate=Coordinate.set_index('QualTime')
print(Spectra.shape)
print(Coordinate.shape)
Spectra = Spectra.iloc[:,0:191].astype('float64')
Spectra.info()
# print(Spectra.index)

In [ ]:
Time=Spectra.index
i = len(Time)-1
while i>=0:
    if(Time[i]<=Time[i-2]+datetime.timedelta(milliseconds=25)):#24 50
        print(i)
        i=i-3
    else:
        print("Removed", i)
        Time=np.delete(Time,i)
        Spectra=Spectra.drop(Spectra.index[i])
        Coordinate=Coordinate.drop(Coordinate.index[i])
        i=i-1

print(Time.shape)
print(Spectra.shape)
print(Coordinate.shape)

In [ ]:
k=2; # we remove this amount of rows for making the number of rows divisible
print(Spectra.shape)
print((len(Spectra)-k)/3)
# len(SpectraNarrow.columns)
# SpectraNarrow[0:len(SpectraNarrow)-2].shape

In [ ]:
n_rows=(len(Spectra)-k)
m_spectra=len(Spectra.columns)
Spectraflat=Spectra[0:n_rows].values.flatten()
print(Spectraflat.shape)
SpectraReshaped = np.reshape(Spectraflat, (int(n_rows/3),3,m_spectra))
print(SpectraReshaped.shape)
SpectraFinal=np.transpose(SpectraReshaped,(0,2,1))
SpectraFinal_flatten=SpectraFinal.flatten()
print(SpectraFinal.shape)
CoordinateFinal=Coordinate[0:n_rows].groupby(np.arange(n_rows)//3).mean()
CoordinateFinal=DataFrame.to_numpy(CoordinateFinal)
print (CoordinateFinal.shape)
print(n_rows/3)

In [ ]:
origin=[375.1,445,204.3]
CoordinateFinal_1=(CoordinateFinal.reshape((-1,20,3))-origin).reshape((-1,60))

[n_tr,m_tr]=CoordinateFinal_1.shape;

y_val_1 = np.zeros((n_tr,60)) # Pre-allocate matrix

for i in range(0,n_tr):
    y_val_1[i,0:3]=CoordinateFinal_1[i,0:3]
    for j in range(0,19):
            y_val_1[i,3*j+3]=CoordinateFinal_1[i,3*j+3]-CoordinateFinal_1[i,3*j]
            y_val_1[i,3*j+4]=CoordinateFinal_1[i,3*j+4]-CoordinateFinal_1[i,3*j+1]
            y_val_1[i,3*j+5]=CoordinateFinal_1[i,3*j+5]-CoordinateFinal_1[i,3*j+2]

In [ ]:
from pickle import dump
from pickle import load


std1 = load(open('scaler.pkl', 'rb')) # load the scalar

x_test_3min_std = std1.transform(SpectraFinal.reshape(-1, 1))
x_test_3min = np.reshape(x_test_3min_std, (-1,190,3))

i=1000
plt.plot(x_test_3min[i,:,:])
fig = plt.figure()

ax= fig.gca(projection='3d')
plt.ion()
fig.show()
xyz=y_val_1[i].reshape((20,3))

ax.plot(xyz[:,0], xyz[:,1], xyz[:,2])

ax.set_box_aspect([1,1,1])
set_axes_equal(ax)

In [ ]:
ScanTime='10-05-2022--15-56'
savetxt('x_3min_Norm_'+ScanTime+'_second.csv', x_test_3min_std, delimiter=';')
savetxt('y_3min_Norm_'+ScanTime+'_second.csv', CoordinateFinal_1, delimiter=';')
avetxt('x_3min_NoPreProcessing_'+ScanTime+'_second.csv', SpectraFinal_flatten, delimiter=';')
savetxt('y_3min_NoPreProcessing_'+ScanTime+'_second.csv', CoordinateFinal_1, delimiter=';')